In [ ]:
pip install gradio groq

In [ ]:
import os
os.environ["GROQ_API_KEY"] = "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"

In [ ]:
# Household Safety Chatbot – Gradio (Colab Stable Compatible)

# -------------------------------
# Install dependencies
# -------------------------------
import subprocess, sys

def install_packages():
    for pkg in ["gradio", "groq"]:
        subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", pkg])

install_packages()

# -------------------------------
# Imports
# -------------------------------
import os
import gradio as gr
from groq import Groq

# -------------------------------
# System Prompt
# -------------------------------
SYSTEM_PROMPT = """You are a Household Safety Expert Chatbot.
ONLY answer household safety related questions.
Politely refuse all off-topic questions.
"""

# -------------------------------
# Chatbot Logic
# -------------------------------
class HouseholdSafetyChatbot:
    def __init__(self):
        api_key = os.getenv("GROQ_API_KEY")
        if not api_key:
            raise ValueError("❌ GROQ_API_KEY not set")

        self.client = Groq(api_key=api_key)
        self.history = []

    def chat(self, message, history):
        if not history:
            self.history = []

        self.history.append({"role": "user", "content": message})
        messages = [{"role": "system", "content": SYSTEM_PROMPT}] + self.history
        response_text = ""

        try:
            stream = self.client.chat.completions.create(
                model="llama-3.3-70b-versatile",
                messages=messages,
                stream=True,
                temperature=0.7,
                max_tokens=1024
            )

            for chunk in stream:
                if chunk.choices[0].delta.content:
                    response_text += chunk.choices[0].delta.content
                    yield self.history + [
                        {"role": "assistant", "content": response_text}
                    ]

            self.history.append(
                {"role": "assistant", "content": response_text}
            )

        except Exception as e:
            yield self.history + [
                {"role": "assistant", "content": f"❌ Error: {e}"}
            ]

    def clear(self):
        self.history = []
        return []

# -------------------------------
# Initialize Bot
# -------------------------------
bot = HouseholdSafetyChatbot()

# -------------------------------
# UI (CORRECT THEME PLACEMENT)
# -------------------------------
with gr.Blocks(
    title="🏠 Household Safety Chatbot",
    theme=gr.themes.Soft()   # ✅ Correct for your version
) as demo:

    gr.Markdown("""
    # 🏠 Household Safety Chatbot
    Ask about fire safety, first aid, childproofing, and emergencies.
    """)

    chat_display = gr.Chatbot(
        height=420,
        type="messages",
        allow_tags=False,   # ✅ Warning fixed
        label="Safety Chat"
    )

    chat_interface = gr.ChatInterface(
        fn=bot.chat,
        chatbot=chat_display,
        type="messages",    # ✅ Warning fixed
        textbox=gr.Textbox(
            placeholder="Ask a household safety question...",
            label="Your Question"
        ),
        examples=[
            "How can I prevent kitchen fires?",
            "What should be in a home first aid kit?",
            "How often should smoke detectors be tested?",
            "How can I prevent carbon monoxide poisoning?"
        ]
    )

    clear_btn = gr.Button("🧹 Clear Chat")
    clear_btn.click(fn=bot.clear, outputs=chat_display)

    gr.Markdown("⚠️ For emergencies, contact local emergency services.")

# -------------------------------
# Launch (NO theme here)
# -------------------------------
demo.launch(share=True, debug=True)

/tmp/ipython-input-3030829015.py:86: DeprecationWarning: The 'theme' parameter in the Blocks constructor will be removed in Gradio 6.0. You will need to pass 'theme' to Blocks.launch() instead.
  with gr.Blocks(
/tmp/ipython-input-3030829015.py:96: DeprecationWarning: The default value of 'allow_tags' in gr.Chatbot will be changed from False to True in Gradio 6.0. You will need to explicitly set allow_tags=False if you want to disable tags in your chatbot.
  chat_display = gr.Chatbot(


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://e01608a84bde1b4e48.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
# Bella
import subprocess
import sys

def install_packages():
    packages = ['gradio', 'groq']
    for package in packages:
        subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", package])

install_packages()

import gradio as gr
from groq import Groq

# System prompt to restrict the chatbot to cooking topics only
SYSTEM_PROMPT = """You are a Cooking Expert Chatbot. Your ONLY purpose is to help users with cooking, food preparation, and culinary-related questions.

Your expertise includes:
- Recipes for various cuisines (Italian, Chinese, Indian, Mexican, French, etc.)
- Cooking techniques and methods (baking, grilling, sautéing, steaming, etc.)
- Ingredient substitutions and alternatives
- Kitchen equipment and tools
- Food preparation and meal planning
- Cooking tips and tricks
- Baking (bread, cakes, pastries, cookies)
- Dietary modifications (vegetarian, vegan, gluten-free, low-carb, etc.)
- Food safety and storage
- Knife skills and cutting techniques
- Seasoning and flavor combinations
- Cooking measurements and conversions
- Sauce and condiment preparation
- Meal prep and batch cooking
- Cooking for special occasions
- Troubleshooting cooking problems
- Plating and presentation
- Regional and international cuisines

STRICT RULES:
1. ONLY answer questions related to cooking, recipes, food preparation, and culinary topics
2. Provide clear, step-by-step instructions when giving recipes
3. Include measurements, cooking times, and temperatures when relevant
4. Suggest alternatives for dietary restrictions when asked
5. If a question is NOT about cooking or food, politely decline and redirect to cooking topics
6. Do NOT answer questions about: politics, technology, sports, entertainment, finance, etc.
7. Be helpful, encouraging, and make cooking accessible to all skill levels
8. When discussing food safety, provide accurate and important safety information

Response format for off-topic questions:
"I apologize, but I'm specifically designed to help with cooking and culinary questions only. I cannot assist with [topic]. However, I'd be happy to help you with recipes, cooking techniques, ingredient substitutions, meal planning, or any other cooking-related questions. What can I help you cook today?"
"""

# Global variables
groq_client = None

def set_api_key(api_key):
    """Initialize Groq client with API key"""
    global groq_client

    if not api_key or api_key.strip() == "":
        return "❌ Please enter a valid Groq API key"

    try:
        groq_client = Groq(api_key=api_key.strip())

        # Test the API key with a simple call
        test_response = groq_client.chat.completions.create(
            model="llama-3.3-70b-versatile",
            messages=[{"role": "user", "content": "Hi"}],
            max_tokens=10
        )

        return "✅ API Key set successfully! You can now start cooking with me!"
    except Exception as e:
        groq_client = None
        return f"❌ Error setting API key: {str(e)}"

def respond(message, history):
    """Process chat messages and return response as a STRING"""
    global groq_client

    # Check if API key is set
    if groq_client is None:
        return "⚠️ Please enter your Groq API key and click 'Set API Key' button first."

    # Check if message is empty
    if not message or message.strip() == "":
        return "⚠️ Please enter a message."

    try:
        # Build messages list for API
        messages = [{"role": "system", "content": SYSTEM_PROMPT}]

        # Add conversation history
        if history:
            for msg in history:
                if isinstance(msg, dict) and "role" in msg and "content" in msg:
                    messages.append({
                        "role": msg["role"],
                        "content": msg["content"]
                    })

        # Add current user message
        messages.append({
            "role": "user",
            "content": message
        })

        # Call Groq API
        completion = groq_client.chat.completions.create(
            model="llama-3.3-70b-versatile",
            messages=messages,
            temperature=0.7,
            max_tokens=2048,
            stream=False
        )

        # Extract and return the response text
        response_text = completion.choices[0].message.content

        return str(response_text)

    except Exception as e:
        error_message = f"❌ Error: {str(e)}\n\nPlease check your API key and try again."
        return str(error_message)

# Create Gradio interface
with gr.Blocks(title="Cooking Chatbot", theme=gr.themes.Soft()) as demo:
    gr.Markdown(
        """
        # 👨‍🍳 Cooking Chatbot
        ### Your Personal Culinary Assistant

        Ask me anything about cooking! Get recipes, cooking tips, ingredient substitutions,
        technique guidance, and more. From beginner basics to advanced culinary techniques!

        **Note:** This chatbot only answers cooking and food preparation questions.
        """
    )

    with gr.Row():
        with gr.Column(scale=3):
            api_key_input = gr.Textbox(
                label="🔑 Groq API Key",
                placeholder="Enter your Groq API key here...",
                type="password"
            )
        with gr.Column(scale=1):
            set_key_btn = gr.Button("Set API Key", variant="primary", size="lg")

    api_status = gr.Textbox(label="Status", interactive=False)

    gr.Markdown("--- ")

    # Create ChatInterface
    chat_interface = gr.ChatInterface(
        fn=respond,
        type="messages",
        chatbot=gr.Chatbot(
            height=450,
            type="messages",
            label="Chat"
        ),
        textbox=gr.Textbox(
            placeholder="Ask me about recipes, cooking techniques, ingredients...",
            label="Your Cooking Question"
        ),
        examples=[
            "How do I make a perfect scrambled eggs?",
            "Give me a recipe for chocolate chip cookies",
            "What can I substitute for eggs in baking?",
            "How do I cook pasta perfectly?",
            "What's a good marinade for chicken?",
            "How do I make homemade pizza dough?",
            "What are the basic knife skills I should learn?",
            "How do I make a simple tomato sauce?",
            "What's the difference between baking soda and baking powder?",
            "Give me a vegetarian lasagna recipe",
            "How do I properly season a cast iron skillet?",
            "What temperature should I cook salmon to?",
        ],
        cache_examples=False
    )

    # Connect the Set API Key button
    set_key_btn.click(
        fn=set_api_key,
        inputs=[api_key_input],
        outputs=[api_status]
    )

    gr.Markdown(
        """
        ---
        ### 📝 Instructions:
        1. **Get your Groq API key**: Visit https://console.groq.com/keys
        2. **Enter the API key** in the field above and click "Set API Key"
        3. **Wait for success message** (✅ API Key set successfully!)
        4. **Start asking** cooking questions

        ### 🍳 What I Can Help You With:
        - **Recipes**: Get detailed recipes for any dish or cuisine
        - **Cooking Techniques**: Learn how to sauté, braise, roast, and more
        - **Ingredient Substitutions**: Find alternatives for dietary needs
        - **Meal Planning**: Plan your weekly meals
        - **Baking**: Master breads, cakes, pastries, and desserts
        - **Food Safety**: Learn proper storage and handling
        - **Kitchen Skills**: Improve your knife work and cooking methods
        - **Dietary Modifications**: Adapt recipes for vegan, gluten-free, etc.
        - **Troubleshooting**: Fix cooking mistakes and problems
        - **International Cuisines**: Explore flavors from around the world

        ### 🍽️ Tips for Best Results:
        - Be specific about serving sizes (e.g., "recipe for 4 people")
        - Mention dietary restrictions upfront
        - Ask follow-up questions for clarification
        - Request step-by-step instructions if needed

        ### ⚠️ Food Safety Reminder:
        Always follow proper food safety guidelines including proper cooking temperatures,
        food storage, and hygiene practices to ensure safe and delicious meals!
        """
    )

# Launch the app
if __name__ == "__main__":
    demo.launch(share=True)

/tmp/ipython-input-1903137281.py:126: DeprecationWarning: The 'theme' parameter in the Blocks constructor will be removed in Gradio 6.0. You will need to pass 'theme' to Blocks.launch() instead.
  with gr.Blocks(title="Cooking Chatbot", theme=gr.themes.Soft()) as demo:
/tmp/ipython-input-1903137281.py:157: DeprecationWarning: The default value of 'allow_tags' in gr.Chatbot will be changed from False to True in Gradio 6.0. You will need to explicitly set allow_tags=False if you want to disable tags in your chatbot.
  chatbot=gr.Chatbot(


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://442147aee496a77507.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:

# Kuppusamy
# Geographic Location Finder Chatbot with Groq API and Gradio
# Designed for Google Colab

# Install required packages
import subprocess
import sys

def install_packages():
    packages = ['gradio', 'groq']
    for package in packages:
        subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", package])

install_packages()

import gradio as gr
from groq import Groq

# System prompt to restrict the chatbot to geographic location topics only
SYSTEM_PROMPT = """You are a Geographic Location Expert Chatbot. Your ONLY purpose is to help users with geographic location information, geography, and place-related questions.

Your expertise includes:
- Countries, capitals, and major cities
- States, provinces, and territories
- Geographic coordinates (latitude and longitude)
- Physical geography (mountains, rivers, lakes, oceans, deserts, etc.)
- Climate and weather patterns by location
- Time zones and regional information
- Population and demographic data of locations
- Famous landmarks and tourist attractions
- Geographic boundaries and borders
- Continents and regions
- Geographic features (plateaus, valleys, peninsulas, etc.)
- Distance and direction between locations
- Cultural and regional characteristics of places
- Geographic trivia and interesting facts about locations
- Map reading and geographic navigation
- Geographic comparisons between locations
- Historical geography and place names
- Geographic terminology and concepts

STRICT RULES:
1. ONLY answer questions related to geographic locations, places, and geography
2. Provide accurate geographic information including coordinates when relevant
3. Help users find and learn about locations worldwide
4. Include context like continent, country, or region when describing locations
5. If a question is NOT about geography or locations, politely decline and redirect
6. Do NOT answer questions about: cooking, sports, entertainment, technology, politics, etc.
7. Be informative and help users understand where places are located
8. Provide interesting geographic facts when relevant

Response format for off-topic questions:
"I apologize, but I'm specifically designed to help with geographic location and geography questions only. I cannot assist with [topic]. However, I'd be happy to help you find locations, learn about countries and cities, discover geographic features, or answer any geography-related questions. What location or geographic information can I help you find today?"
"""

# Global variables
groq_client = None

def set_api_key(api_key):
    """Initialize Groq client with API key"""
    global groq_client

    if not api_key or api_key.strip() == "":
        return "❌ Please enter a valid Groq API key"

    try:
        groq_client = Groq(api_key=api_key.strip())

        # Test the API key with a simple call
        test_response = groq_client.chat.completions.create(
            model="llama-3.3-70b-versatile",
            messages=[{"role": "user", "content": "Hi"}],
            max_tokens=10
        )

        return "✅ API Key set successfully! You can now explore geographic locations!"
    except Exception as e:
        groq_client = None
        return f"❌ Error setting API key: {str(e)}"

def respond(message, history):
    """Process chat messages and return response as a STRING"""
    global groq_client

    # Check if API key is set
    if groq_client is None:
        return "⚠️ Please enter your Groq API key and click 'Set API Key' button first."

    # Check if message is empty
    if not message or message.strip() == "":
        return "⚠️ Please enter a message."

    try:
        # Build messages list for API
        messages = [{"role": "system", "content": SYSTEM_PROMPT}]

        # Add conversation history
        if history:
            for msg in history:
                if isinstance(msg, dict) and "role" in msg and "content" in msg:
                    messages.append({
                        "role": msg["role"],
                        "content": msg["content"]
                    })

        # Add current user message
        messages.append({
            "role": "user",
            "content": message
        })

        # Call Groq API
        completion = groq_client.chat.completions.create(
            model="llama-3.3-70b-versatile",
            messages=messages,
            temperature=0.7,
            max_tokens=1536,
            stream=False
        )

        # Extract and return the response text
        response_text = completion.choices[0].message.content

        return str(response_text)

    except Exception as e:
        error_message = f"❌ Error: {str(e)}\n\nPlease check your API key and try again."
        return str(error_message)

# Create Gradio interface
with gr.Blocks(title="Geographic Location Finder Chatbot", theme=gr.themes.Soft()) as demo:
    gr.Markdown(
        """
        # 🌍 Geographic Location Finder Chatbot
        ### Your Personal Geography Assistant

        Ask me anything about geographic locations! Find countries, cities, landmarks,
        coordinates, geographic features, and explore the world through geography.

        **Note:** This chatbot only answers geography and location-related questions.
        """
    )

    with gr.Row():
        with gr.Column(scale=3):
            api_key_input = gr.Textbox(
                label="🔑 Groq API Key",
                placeholder="Enter your Groq API key here...",
                type="password"
            )
        with gr.Column(scale=1):
            set_key_btn = gr.Button("Set API Key", variant="primary", size="lg")

    api_status = gr.Textbox(label="Status", interactive=False)

    gr.Markdown("--- ")

    # Create ChatInterface
    chat_interface = gr.ChatInterface(
        fn=respond,
        type="messages",
        chatbot=gr.Chatbot(
            height=450,
            type="messages",
            label="Chat"9
        ),
        textbox=gr.Textbox(
            placeholder="Ask about any location, city, country, or geographic feature...",
            label="Your Geography Question"
        ),
        examples=[
            "What is the capital of France?",
            "Where is Mount Everest located?",
            "What are the coordinates of the Eiffel Tower?",
            "Tell me about the Amazon River",
            "What countries border Germany?",
            "Where is the Great Barrier Reef?",
            "What is the highest mountain in Africa?",
            "Which time zone is New York in?",
            "What are the largest cities in Japan?",
            "Where is the Sahara Desert located?",
            "What is the distance between London and Paris?",
            "Tell me about the Grand Canyon",
            "What continent is Brazil on?",
            "Where are the Maldives located?",
            "What is the population of Tokyo?",
        ],
        cache_examples=False
    )

    # Connect the Set API Key button
    set_key_btn.click(
        fn=set_api_key,
        inputs=[api_key_input],
        outputs=[api_status]
    )

    gr.Markdown(
        """
        ---
        ### 📝 Instructions:
        1. **Get your Groq API key**: Visit https://console.groq.com/keys
        2. **Enter the API key** in the field above and click "Set API Key"
        3. **Wait for success message** (✅ API Key set successfully!)
        4. **Start exploring** geographic locations

        ### 🗺️ What I Can Help You Find:
        - **Countries & Capitals**: Information about nations and their capitals
        - **Cities & Towns**: Major and minor cities worldwide
        - **Geographic Coordinates**: Latitude and longitude of any location
        - **Physical Features**: Mountains, rivers, lakes, oceans, deserts
        - **Landmarks**: Famous monuments, buildings, and tourist attractions
        - **Boundaries & Borders**: Which countries share borders
        - **Climate Zones**: Weather patterns and climate information
        - **Time Zones**: What time zone different locations are in
        - **Population Data**: Demographics of cities and countries
        - **Distances**: How far apart different locations are
        - **Continents & Regions**: Regional groupings and characteristics
        - **Cultural Geography**: Regional characteristics and traits

        ### 🌟 Types of Questions You Can Ask:
        - "Where is [place]?"
        - "What is the capital of [country]?"
        - "What are the coordinates of [location]?"
        - "Tell me about [geographic feature]"
        - "What countries are in [region]?"
        - "How far is [place A] from [place B]?"
        - "What is the highest/largest/longest [feature] in [location]?"
        - "What time zone is [city] in?"

        ### 🎯 Perfect For:
        - Students studying geography
        - Travelers planning trips
        - Geography enthusiasts
        - Anyone curious about the world
        - Educational purposes
        - Geographic research
        """
    )

# Launch the app
if __name__ == "__main__":
    demo.launch(share=True)

/tmp/ipython-input-3602762432.py:130: DeprecationWarning: The 'theme' parameter in the Blocks constructor will be removed in Gradio 6.0. You will need to pass 'theme' to Blocks.launch() instead.
  with gr.Blocks(title="Geographic Location Finder Chatbot", theme=gr.themes.Soft()) as demo:
/tmp/ipython-input-3602762432.py:161: DeprecationWarning: The default value of 'allow_tags' in gr.Chatbot will be changed from False to True in Gradio 6.0. You will need to explicitly set allow_tags=False if you want to disable tags in your chatbot.
  chatbot=gr.Chatbot(


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://a9ca3f718464018cc1.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:

# Jency Diana
# Business English Chatbot with Groq API and Gradio
# Designed for Google Colab

# Install required packages
import subprocess
import sys

def install_packages():
    packages = ['gradio', 'groq']
    for package in packages:
        subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", package])

install_packages()

import gradio as gr
from groq import Groq

# System prompt to restrict the chatbot to business English topics only
SYSTEM_PROMPT = """You are a Business English Expert Chatbot. Your ONLY purpose is to help users learn, practice, and improve their Business English skills.

Your expertise includes:
- Business vocabulary and terminology
- Professional email writing and formatting
- Business letter and report writing
- Meeting language and phrases
- Presentation skills and language
- Negotiation language and techniques
- Phone and video call etiquette
- Business idioms and expressions
- Formal vs. informal business language
- Corporate communication styles
- Job interview preparation and language
- Resume and cover letter writing
- Business correspondence templates
- Cross-cultural business communication
- Professional networking language
- Sales and marketing language
- Financial and accounting terminology
- HR and recruitment language
- Contract and legal terminology
- Business proposals and pitches
- Small talk and relationship-building language
- Apologizing and handling conflicts professionally
- Making requests and suggestions politely
- Grammar and punctuation for business writing

STRICT RULES:
1. ONLY answer questions related to Business English, professional communication, and workplace language
2. Provide clear examples and templates when relevant
3. Explain formal vs. informal usage in business contexts
4. Correct grammar and suggest more professional alternatives when appropriate
5. Help users sound more professional and confident in English
6. If a question is NOT about Business English or professional communication, politely decline and redirect
7. Do NOT answer questions about: general topics, entertainment, sports, cooking, personal matters, etc.
8. Focus on practical, workplace-applicable language skills
9. Provide cultural context for business communication when helpful

Response format for off-topic questions:
"I apologize, but I'm specifically designed to help with Business English and professional communication only. I cannot assist with [topic]. However, I'd be happy to help you with business emails, professional vocabulary, meeting language, presentation skills, or any other Business English topic. What Business English skill can I help you with today?"
"""

# Global variables
groq_client = None

def set_api_key(api_key):
    """Initialize Groq client with API key"""
    global groq_client

    if not api_key or api_key.strip() == "":
        return "❌ Please enter a valid Groq API key"

    try:
        groq_client = Groq(api_key=api_key.strip())

        # Test the API key with a simple call
        test_response = groq_client.chat.completions.create(
            model="llama-3.3-70b-versatile",
            messages=[{"role": "user", "content": "Hi"}],
            max_tokens=10
        )

        return "✅ API Key set successfully! Let's improve your Business English!"
    except Exception as e:
        groq_client = None
        return f"❌ Error setting API key: {str(e)}"

def respond(message, history):
    """Process chat messages and return response as a STRING"""
    global groq_client

    # Check if API key is set
    if groq_client is None:
        return "⚠️ Please enter your Groq API key and click 'Set API Key' button first."

    # Check if message is empty
    if not message or message.strip() == "":
        return "⚠️ Please enter a message."

    try:
        # Build messages list for API
        messages = [{"role": "system", "content": SYSTEM_PROMPT}]

        # Add conversation history
        if history:
            for msg in history:
                if isinstance(msg, dict) and "role" in msg and "content" in msg:
                    messages.append({
                        "role": msg["role"],
                        "content": msg["content"]
                    })

        # Add current user message
        messages.append({
            "role": "user",
            "content": message
        })

        # Call Groq API
        completion = groq_client.chat.completions.create(
            model="llama-3.3-70b-versatile",
            messages=messages,
            temperature=0.7,
            max_tokens=2048,
            stream=False
        )

        # Extract and return the response text
        response_text = completion.choices[0].message.content

        return str(response_text)

    except Exception as e:
        error_message = f"❌ Error: {str(e)}\n\nPlease check your API key and try again."
        return str(error_message)

# Create Gradio interface
with gr.Blocks(title="Business English Chatbot", theme=gr.themes.Soft()) as demo:
    gr.Markdown(
        """
        # 💼 Business English Chatbot
        ### Your Professional Communication Assistant

        Master Business English with AI assistance! Get help with professional emails,
        business vocabulary, meeting language, presentations, and more.

        **Note:** This chatbot focuses exclusively on Business English and professional communication.
        """
    )

    with gr.Row():
        with gr.Column(scale=3):
            api_key_input = gr.Textbox(
                label="🔑 Groq API Key",
                placeholder="Enter your Groq API key here...",
                type="password"
            )
        with gr.Column(scale=1):
            set_key_btn = gr.Button("Set API Key", variant="primary", size="lg")

    api_status = gr.Textbox(label="Status", interactive=False)

    gr.Markdown("---")

    # Create ChatInterface
    chat_interface = gr.ChatInterface(
        fn=respond,
        type="messages",
        chatbot=gr.Chatbot(
            height=450,
            type="messages",
            label="Chat",
            allow_tags=False # Explicitly set allow_tags to False
        ),
        textbox=gr.Textbox(
            placeholder="Ask about business emails, professional vocabulary, presentations...",
            label="Your Business English Question"
        ),
        examples=[
            "How do I write a professional email to request a meeting?",
            "What are common phrases for presentations?",
            "Help me write a follow-up email after an interview",
            "What's the difference between 'regards' and 'best regards'?",
            "How do I politely decline a business invitation?",
            "What are useful phrases for negotiations?",
            "How do I introduce myself in a business meeting?",
            "Write a template for a formal complaint email",
            "What are common business idioms I should know?",
            "How do I apologize professionally in an email?",
            "What phrases can I use for making suggestions in meetings?",
            "Help me write a professional out-of-office message",
            "What's the proper way to address someone in a business email?",
            "Give me phrases for agreeing and disagreeing politely",
            "How do I write a professional LinkedIn message?",
        ],
        cache_examples=False
    )

    # Connect the Set API Key button
    set_key_btn.click(
        fn=set_api_key,
        inputs=[api_key_input],
        outputs=[api_status]
    )

    gr.Markdown(
        """
        ---
        ### 📝 Instructions:
        1. **Get your Groq API key**: Visit https://console.groq.com/keys
        2. **Enter the API key** in the field above and click "Set API Key"
        3. **Wait for success message** (✅ API Key set successfully!)
        4. **Start improving** your Business English skills

        ### 📧 What I Can Help You With:

        **Written Communication:**
        - Professional email writing (requests, confirmations, apologies, etc.)
        - Business letters and reports
        - Resume and cover letter writing
        - Proposals and business documents
        - Formal vs. informal language choices

        **Spoken Communication:**
        - Meeting language and phrases
        - Presentation skills and structure
        - Phone and video call etiquette
        - Negotiation language
        - Job interview preparation
        - Small talk and networking

        **Vocabulary & Grammar:**
        - Business terminology and jargon
        - Industry-specific vocabulary
        - Business idioms and expressions
        - Grammar for professional writing
        - Punctuation and formatting

        **Professional Skills:**
        - Cross-cultural communication
        - Handling conflicts professionally
        - Making polite requests
        - Giving and receiving feedback
        - Professional networking language

        ### 💡 How to Use This Chatbot:

        **For Email Writing:**
        - "Help me write an email to [purpose]"
        - "Review this email: [paste your email]"
        - "What's a professional way to say [phrase]?"

        **For Learning Phrases:**
        - "What phrases can I use for [situation]?"
        - "Give me common expressions for [context]"
        - "How do I [action] professionally?"

        **For Practice:**
        - Ask for templates and examples
        - Request corrections and improvements
        - Learn alternatives for common phrases

        ### 🎯 Perfect For:
        - **Professionals** improving workplace communication
        - **Non-native English speakers** in business environments
        - **Job seekers** preparing applications and interviews
        - **Students** learning Business English
        - **Entrepreneurs** communicating with clients
        - **Anyone** wanting to sound more professional

        ### ✨ Tips for Success:
        - Be specific about your context (formal/informal, audience, purpose)
        - Ask for multiple options or alternatives
        - Request explanations for why certain phrases are more professional
        - Practice with real workplace scenarios
        """
    )

# Launch the app
if __name__ == "__main__":
    demo.launch(share=True)

/tmp/ipython-input-2646541775.py:137: DeprecationWarning: The 'theme' parameter in the Blocks constructor will be removed in Gradio 6.0. You will need to pass 'theme' to Blocks.launch() instead.
  with gr.Blocks(title="Business English Chatbot", theme=gr.themes.Soft()) as demo:
/tmp/ipython-input-2646541775.py:168: DeprecationWarning: The default value of 'allow_tags' in gr.Chatbot will be changed from False to True in Gradio 6.0. You will need to explicitly set allow_tags=False if you want to disable tags in your chatbot.
  chatbot=gr.Chatbot(


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://b47868092ca7301395.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
# Suyambu Kesavan
# Road Safety Rules Chatbot with Groq API and Gradio
# Designed for Google Colab

# Install required packages
import subprocess
import sys

def install_packages():
    packages = ['gradio', 'groq']
    for package in packages:
        subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", package])

install_packages()

import gradio as gr
from groq import Groq

# System prompt to restrict the chatbot to road safety topics only
SYSTEM_PROMPT = """You are a Road Safety Expert Chatbot. Your ONLY purpose is to help users understand and learn about road safety rules, traffic regulations, and safe driving practices.

Your expertise includes:
- Traffic rules and regulations
- Road signs and their meanings (warning, regulatory, informational signs)
- Traffic signals and light rules
- Right-of-way rules and priorities
- Speed limits and safe driving speeds
- Lane discipline and lane changing rules
- Pedestrian safety and crosswalk rules
- Bicycle and motorcycle safety
- Parking rules and regulations
- Driving in different weather conditions (rain, fog, snow)
- Night driving safety
- Highway and expressway rules
- Roundabout and intersection navigation
- School zone safety
- Drunk driving laws and dangers
- Distracted driving prevention
- Seat belt and child safety seat regulations
- Vehicle maintenance for safety
- Emergency vehicle protocols
- Defensive driving techniques
- Accident prevention strategies
- Road safety for children and elderly
- Vehicle blind spots and mirror usage
- Safe following distances
- Overtaking and passing rules

STRICT RULES:
1. ONLY answer questions related to road safety, traffic rules, and driving regulations
2. Provide clear, accurate, and safety-focused information
3. Emphasize the importance of following traffic laws
4. Give practical tips for safe driving and road usage
5. Explain the reasoning behind safety rules when helpful
6. If a question is NOT about road safety, politely decline and redirect to road safety topics
7. Do NOT answer questions about: entertainment, cooking, politics, sports, general knowledge, etc.
8. Always prioritize safety in your responses
9. Encourage responsible and defensive driving behavior
10. When discussing laws, note that specific regulations may vary by country/region

Response format for off-topic questions:
"I apologize, but I'm specifically designed to help with road safety rules and traffic regulations only. I cannot assist with [topic]. However, I'd be happy to help you with questions about traffic signs, safe driving practices, road rules, pedestrian safety, or any other road safety topic. What road safety question can I help you with today?"
"""

# Global variables
groq_client = None

def set_api_key(api_key):
    """Initialize Groq client with API key"""
    global groq_client

    if not api_key or api_key.strip() == "":
        return "❌ Please enter a valid Groq API key"

    try:
        groq_client = Groq(api_key=api_key.strip())

        # Test the API key with a simple call
        test_response = groq_client.chat.completions.create(
            model="llama-3.3-70b-versatile",
            messages=[{"role": "user", "content": "Hi"}],
            max_tokens=10
        )

        return "✅ API Key set successfully! Let's learn about road safety!"
    except Exception as e:
        groq_client = None
        return f"❌ Error setting API key: {str(e)}"

def respond(message, history):
    """Process chat messages and return response as a STRING"""
    global groq_client

    # Check if API key is set
    if groq_client is None:
        return "⚠️ Please enter your Groq API key and click 'Set API Key' button first."

    # Check if message is empty
    if not message or message.strip() == "":
        return "⚠️ Please enter a message."

    try:
        # Build messages list for API
        messages = [{"role": "system", "content": SYSTEM_PROMPT}]

        # Add conversation history
        if history:
            for msg in history:
                if isinstance(msg, dict) and "role" in msg and "content" in msg:
                    messages.append({
                        "role": msg["role"],
                        "content": msg["content"]
                    })

        # Add current user message
        messages.append({
            "role": "user",
            "content": message
        })

        # Call Groq API
        completion = groq_client.chat.completions.create(
            model="llama-3.3-70b-versatile",
            messages=messages,
            temperature=0.7,
            max_tokens=1536,
            stream=False
        )

        # Extract and return the response text
        response_text = completion.choices[0].message.content

        return str(response_text)

    except Exception as e:
        error_message = f"❌ Error: {str(e)}\n\nPlease check your API key and try again."
        return str(error_message)

# Create Gradio interface
with gr.Blocks(title="Road Safety Rules Chatbot", theme=gr.themes.Soft()) as demo:
    gr.Markdown(
        """
        # 🚦 Road Safety Rules Chatbot
        ### Your Personal Road Safety Assistant

        Learn about traffic rules, road signs, safe driving practices, and road safety regulations.
        Stay safe on the roads with expert guidance!

        **Note:** This chatbot only answers road safety and traffic rule questions.
        """
    )

    with gr.Row():
        with gr.Column(scale=3):
            api_key_input = gr.Textbox(
                label="🔑 Groq API Key",
                placeholder="Enter your Groq API key here...",
                type="password"
            )
        with gr.Column(scale=1):
            set_key_btn = gr.Button("Set API Key", variant="primary", size="lg")

    api_status = gr.Textbox(label="Status", interactive=False)

    gr.Markdown("---")

    # Create ChatInterface
    chat_interface = gr.ChatInterface(
        fn=respond,
        type="messages",
        chatbot=gr.Chatbot(
            height=450,
            type="messages",
            label="Chat",
            allow_tags=False
        ),
        textbox=gr.Textbox(
            placeholder="Ask about traffic rules, road signs, safe driving...",
            label="Your Road Safety Question"
        ),
        examples=[
            "What does a red octagonal sign mean?",
            "What are the rules for using a roundabout?",
            "How should I drive safely in heavy rain?",
            "What is the safe following distance behind another vehicle?",
            "Explain the rules for pedestrian crossings",
            "What do yellow traffic lights mean?",
            "What are the speed limit rules in school zones?",
            "How do I safely change lanes on a highway?",
            "What should I do when an emergency vehicle approaches?",
            "What are the rules for wearing seat belts?",
            "How can I avoid distracted driving?",
            "What are the dangers of drunk driving?",
            "How do I navigate a four-way stop intersection?",
            "What are blind spots and how do I check them?",
            "What are the rules for overtaking other vehicles?",
        ],
        cache_examples=False
    )

    # Connect the Set API Key button
    set_key_btn.click(
        fn=set_api_key,
        inputs=[api_key_input],
        outputs=[api_status]
    )

    gr.Markdown(
        """
        ---
        ### 📝 Instructions:
        1. **Get your Groq API key**: Visit https://console.groq.com/keys
        2. **Enter the API key** in the field above and click "Set API Key"
        3. **Wait for success message** (✅ API Key set successfully!)
        4. **Start learning** about road safety rules

        ### 🚗 What I Can Help You With:

        **Traffic Rules & Regulations:**
        - Speed limits and safe driving speeds
        - Right-of-way rules
        - Lane discipline and changing
        - Parking regulations
        - Intersection and roundabout rules
        - Overtaking and passing rules

        **Road Signs & Signals:**
        - Warning signs (yellow/diamond)
        - Regulatory signs (red/white)
        - Informational signs (blue/green)
        - Traffic light meanings
        - Road markings and pavement signs

        **Safe Driving Practices:**
        - Defensive driving techniques
        - Weather-specific driving (rain, fog, snow)
        - Night driving safety
        - Highway driving rules
        - Safe following distances
        - Mirror and blind spot checks

        **Special Situations:**
        - School zone safety
        - Emergency vehicle protocols
        - Construction zone navigation
        - Pedestrian and bicycle safety
        - Motorcycle safety

        **Vehicle Safety:**
        - Seat belt regulations
        - Child safety seats
        - Vehicle maintenance for safety
        - Tire and brake safety

        **Risk Prevention:**
        - Avoiding distracted driving
        - Drunk driving dangers
        - Fatigue and drowsy driving
        - Aggressive driving prevention
        - Accident prevention strategies

        ### 🎯 Perfect For:
        - **New Drivers** - Learning the basics of road safety
        - **License Applicants** - Preparing for driving tests
        - **Parents** - Teaching children about road safety
        - **Experienced Drivers** - Refreshing knowledge
        - **Pedestrians & Cyclists** - Understanding road rules
        - **Driving Instructors** - Educational resource

        ### ⚠️ Important Reminders:
        - Traffic laws may vary by country, state, or region
        - Always follow local traffic regulations
        - When in doubt, drive cautiously and defensively
        - Never drive under the influence of alcohol or drugs
        - Always wear your seat belt
        - Avoid using mobile phones while driving

        ### 🚨 Emergency Contacts:
        - In case of accidents, call emergency services immediately
        - Report dangerous driving behavior to local authorities
        - For road safety emergencies, prioritize safety first

        **Stay safe on the roads! Drive smart, arrive safe.** 🚦🚗✨
        """
    )

# Launch the app
if __name__ == "__main__":
    demo.launch(share=True)

/tmp/ipython-input-3458775162.py:139: DeprecationWarning: The 'theme' parameter in the Blocks constructor will be removed in Gradio 6.0. You will need to pass 'theme' to Blocks.launch() instead.
  with gr.Blocks(title="Road Safety Rules Chatbot", theme=gr.themes.Soft()) as demo:
/tmp/ipython-input-3458775162.py:170: DeprecationWarning: The default value of 'allow_tags' in gr.Chatbot will be changed from False to True in Gradio 6.0. You will need to explicitly set allow_tags=False if you want to disable tags in your chatbot.
  chatbot=gr.Chatbot(


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://e6d665f34dfb0a7e8a.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
# Physics Course Chatbot with Groq API and Gradio
# Designed for Google Colab

# Install required packages
import subprocess
import sys

def install_packages():
    packages = ['gradio', 'groq']
    for package in packages:
        subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", package])

install_packages()

import gradio as gr
from groq import Groq

# System prompt to restrict the chatbot to physics topics only
SYSTEM_PROMPT = """You are a Physics Course Expert Chatbot. Your ONLY purpose is to help users learn, understand, and master physics concepts from high school to university level.

Your expertise includes:

**Classical Mechanics:**
- Kinematics (motion, velocity, acceleration)
- Newton's Laws of Motion
- Force, mass, and momentum
- Work, energy, and power
- Conservation laws (energy, momentum, angular momentum)
- Circular motion and gravitation
- Rotational dynamics
- Simple harmonic motion and oscillations

**Thermodynamics:**
- Temperature, heat, and thermal energy
- Laws of thermodynamics
- Heat transfer (conduction, convection, radiation)
- Ideal gas laws
- Entropy and statistical mechanics
- Heat engines and refrigerators

**Electromagnetism:**
- Electric charge and Coulomb's law
- Electric fields and potential
- Current, resistance, and Ohm's law
- Magnetic fields and forces
- Electromagnetic induction
- Maxwell's equations
- AC/DC circuits

**Waves and Optics:**
- Wave properties (wavelength, frequency, amplitude)
- Sound waves and acoustics
- Light and electromagnetic waves
- Reflection, refraction, and diffraction
- Interference and polarization
- Geometric and wave optics
- Doppler effect

**Modern Physics:**
- Quantum mechanics basics
- Atomic structure and spectra
- Photoelectric effect
- Wave-particle duality
- Relativity (special and general)
- Nuclear physics
- Particle physics fundamentals

**Additional Topics:**
- Fluid mechanics
- Dimensional analysis
- Problem-solving strategies
- Mathematical methods in physics
- Laboratory techniques
- Physics applications in real world

STRICT RULES:
1. ONLY answer questions related to physics concepts, theories, problems, and applications
2. Provide clear, step-by-step explanations with equations when relevant
3. Use proper physics notation and units (SI units preferred)
4. Help students understand concepts, not just memorize formulas
5. Break down complex problems into manageable steps
6. If a question is NOT about physics, politely decline and redirect to physics topics
7. Do NOT answer questions about: cooking, entertainment, politics, sports, unrelated subjects
8. Encourage conceptual understanding alongside mathematical solutions
9. Provide real-world examples and applications when helpful
10. Explain the physical intuition behind mathematical formulas

Response format for off-topic questions:
"I apologize, but I'm specifically designed to help with physics course topics only. I cannot assist with [topic]. However, I'd be happy to help you with mechanics, thermodynamics, electromagnetism, optics, quantum physics, problem-solving, or any other physics-related questions. What physics concept can I help you understand today?"
"""

# Global variables
groq_client = None

def set_api_key(api_key):
    """Initialize Groq client with API key"""
    global groq_client

    if not api_key or api_key.strip() == "":
        return "❌ Please enter a valid Groq API key"

    try:
        groq_client = Groq(api_key=api_key.strip())

        # Test the API key with a simple call
        test_response = groq_client.chat.completions.create(
            model="llama-3.3-70b-versatile",
            messages=[{"role": "user", "content": "Hi"}],
            max_tokens=10
        )

        return "✅ API Key set successfully! Let's explore physics together!"
    except Exception as e:
        groq_client = None
        return f"❌ Error setting API key: {str(e)}"

def respond(message, history):
    """Process chat messages and return response as a STRING"""
    global groq_client

    # Check if API key is set
    if groq_client is None:
        return "⚠️ Please enter your Groq API key and click 'Set API Key' button first."

    # Check if message is empty
    if not message or message.strip() == "":
        return "⚠️ Please enter a message."

    try:
        # Build messages list for API
        messages = [{"role": "system", "content": SYSTEM_PROMPT}]

        # Add conversation history
        if history:
            for msg in history:
                if isinstance(msg, dict) and "role" in msg and "content" in msg:
                    messages.append({
                        "role": msg["role"],
                        "content": msg["content"]
                    })

        # Add current user message
        messages.append({
            "role": "user",
            "content": message
        })

        # Call Groq API
        completion = groq_client.chat.completions.create(
            model="llama-3.3-70b-versatile",
            messages=messages,
            temperature=0.7,
            max_tokens=2048,
            stream=False
        )

        # Extract and return the response text
        response_text = completion.choices[0].message.content

        return str(response_text)

    except Exception as e:
        error_message = f"❌ Error: {str(e)}\n\nPlease check your API key and try again."
        return str(error_message)

# Create Gradio interface
with gr.Blocks(title="Physics Course Chatbot", theme=gr.themes.Soft()) as demo:
    gr.Markdown(
        """
        # ⚛️ Physics Course Chatbot
        ### Your Personal Physics Tutor

        Master physics from classical mechanics to quantum theory! Get help with concepts,
        problem-solving, equations, and real-world applications.

        **Note:** This chatbot only answers physics course-related questions.
        """
    )

    with gr.Row():
        with gr.Column(scale=3):
            api_key_input = gr.Textbox(
                label="🔑 Groq API Key",
                placeholder="Enter your Groq API key here...",
                type="password"
            )
        with gr.Column(scale=1):
            set_key_btn = gr.Button("Set API Key", variant="primary", size="lg")

    api_status = gr.Textbox(label="Status", interactive=False)

    gr.Markdown("---")

    # Create ChatInterface
    chat_interface = gr.ChatInterface(
        fn=respond,
        type="messages",
        chatbot=gr.Chatbot(
            height=450,
            type="messages",
            label="Chat"
        ),
        textbox=gr.Textbox(
            placeholder="Ask about physics concepts, problems, equations...",
            label="Your Physics Question"
        ),
        examples=[
            "Explain Newton's Second Law of Motion",
            "What is the difference between speed and velocity?",
            "How do I calculate kinetic energy?",
            "Explain the First Law of Thermodynamics",
            "What is Ohm's Law and how do I use it?",
            "Explain the photoelectric effect",
            "What are Maxwell's equations?",
            "How does simple harmonic motion work?",
            "Explain the concept of electric potential",
            "What is the difference between heat and temperature?",
            "How do I solve projectile motion problems?",
            "Explain wave-particle duality",
            "What is conservation of momentum?",
            "How does electromagnetic induction work?",
            "Explain Einstein's theory of special relativity",
        ],
        cache_examples=False
    )

    # Connect the Set API Key button
    set_key_btn.click(
        fn=set_api_key,
        inputs=[api_key_input],
        outputs=[api_status]
    )

    gr.Markdown(
        """
        ---
        ### 📝 Instructions:
        1. **Get your Groq API key**: Visit https://console.groq.com/keys
        2. **Enter the API key** in the field above and click "Set API Key"
        3. **Wait for success message** (✅ API Key set successfully!)
        4. **Start learning** physics concepts

        ### 📚 Physics Topics Covered:

        **Classical Mechanics:**
        - Kinematics and dynamics
        - Newton's Laws
        - Work, energy, and power
        - Momentum and collisions
        - Circular motion and gravitation
        - Rotational mechanics
        - Oscillations and waves

        **Thermodynamics:**
        - Temperature and heat
        - Laws of thermodynamics
        - Heat transfer mechanisms
        - Ideal gases and kinetic theory
        - Entropy and thermodynamic cycles

        **Electromagnetism:**
        - Electric fields and forces
        - Electric potential and circuits
        - Magnetic fields and forces
        - Electromagnetic induction
        - AC/DC circuits analysis
        - Maxwell's equations

        **Waves and Optics:**
        - Wave properties and behavior
        - Sound and acoustics
        - Light and electromagnetic spectrum
        - Reflection and refraction
        - Interference and diffraction
        - Optical instruments

        **Modern Physics:**
        - Quantum mechanics fundamentals
        - Atomic and nuclear physics
        - Photoelectric effect
        - Wave-particle duality
        - Special and general relativity
        - Particle physics basics

        **Problem-Solving Skills:**
        - Step-by-step solutions
        - Dimensional analysis
        - Free body diagrams
        - Conservation laws application
        - Mathematical methods

        ### 💡 How to Get the Most Out of This Chatbot:

        **For Concept Understanding:**
        - "Explain [concept] in simple terms"
        - "What is the physical meaning of [formula]?"
        - "Give me an example of [physics principle]"

        **For Problem Solving:**
        - "How do I solve this problem: [describe problem]"
        - "Walk me through the steps for [type of problem]"
        - "What approach should I use for [physics scenario]?"

        **For Equations and Formulas:**
        - "What is the formula for [quantity]?"
        - "When do I use [equation]?"
        - "Derive the equation for [physics concept]"

        **For Applications:**
        - "What are real-world applications of [concept]?"
        - "How is [physics principle] used in technology?"
        - "Give me examples where [law] applies"

        ### 🎯 Perfect For:
        - **High School Students** - AP Physics, IB Physics
        - **University Students** - Physics 101, 102, and beyond
        - **Engineering Students** - Physics fundamentals
        - **Self-Learners** - Understanding physics independently
        - **Exam Preparation** - Homework help and test prep
        - **Teachers** - Explaining concepts differently

        ### 🔬 Learning Tips:
        - Start with conceptual understanding before diving into math
        - Draw diagrams to visualize problems
        - Practice dimensional analysis to check your work
        - Understand units and SI system
        - Connect physics concepts to everyday phenomena
        - Work through problems step-by-step

        ### 📐 Important Notes:
        - All units follow SI system unless specified
        - Equations use standard physics notation
        - Solutions emphasize understanding over memorization
        - Real-world applications enhance learning
        - Mathematical rigor balanced with intuition

        **Ready to master physics? Let's explore the universe together!** ⚛️🔬🌌
        """
    )

# Launch the app
if __name__ == "__main__":
    demo.launch(share=True)

/tmp/ipython-input-1157371702.py:167: DeprecationWarning: The 'theme' parameter in the Blocks constructor will be removed in Gradio 6.0. You will need to pass 'theme' to Blocks.launch() instead.
  with gr.Blocks(title="Physics Course Chatbot", theme=gr.themes.Soft()) as demo:
/tmp/ipython-input-1157371702.py:198: DeprecationWarning: The default value of 'allow_tags' in gr.Chatbot will be changed from False to True in Gradio 6.0. You will need to explicitly set allow_tags=False if you want to disable tags in your chatbot.
  chatbot=gr.Chatbot(


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://cf2cffa9202600093b.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
